## Captação dos dados VIA API

Utilizando NewsAPI para fazer resgate das notícias dos últimos 30 dias

In [18]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Obter a data atual
data_atual = datetime.now()
data = data_atual - timedelta(days=60)
data_formatada = data.strftime("%Y-%m-%d")

# URL da API
url = (f'https://newsapi.org/v2/everything?'
       'q=agronegócio&'
       'from={data_formatada}'
       'sortBy=popularity&'
       'apiKey=df22a809929b459796eda53179209ec0')

# Fazendo a requisição; making the requisition
response = requests.get(url)

if response.status_code == 200:
    noticias = response.json()
    articles = noticias.get("articles", [])
    
    dados = []

    if articles:
        for artigo in articles:
            dados.append({
                "Título": artigo.get("title", "N/A"),
                "Descrição": artigo.get("description", "N/A"),
                "Data de Publicação": artigo.get("publishedAt", "N/A"),
                "URL": artigo.get("url", "N/A"),
                "Fonte": artigo.get("source", {}).get("name", "N/A")  # Fonte do artigo; article font
            })
        
        df = pd.DataFrame(dados)
        display(df.head(10))
    else:
        print("Nenhuma notícia encontrada.")
else:
    print(f"Erro: {response.status_code}, {response.json().get('message')}")


,Título,Descrição,Data de Publicação,URL,Fonte
0,Primeiro satélite brasileiro lançado em territ...,Empresa que coloca IA no agronegócio financia ...,2024-11-15T17:48:25Z,https://olhardigital.com.br/2024/11/15/ciencia...,Olhardigital.com.br
1,"‘O mundo precisa do Brasil’, diz Roberto Rodri...","Na avaliação de Rodrigues, agronegócio brasil...",2024-12-13T09:00:47Z,https://veja.abril.com.br/economia/o-mundo-pre...,Abril.com.br
2,[Removed],[Removed],2024-11-18T21:01:00Z,https://removed.com,[Removed]
3,Crise das carnes: agronegócio espera retrataçã...,Alexandre Bompard questionou qualidade do prod...,2024-11-25T20:51:35Z,https://veja.abril.com.br/coluna/radar-economi...,Abril.com.br
4,Agronegócio: Veja as novas tarifas após acordo...,Assinado nesta sexta-feira (6) após 25 anos de...,2024-12-08T15:19:34Z,https://dinheirama.com/agronegocio-veja-as-nov...,Dinheirama.com
5,Bancada do agro confirma apoio a Hugo Motta à ...,O presidente da Frente Parlamentar do Agronegó...,2024-12-03T16:44:40Z,https://www.terra.com.br/noticias/brasil/polit...,Terra.com.br
6,A mais nova parceira do agronegócio brasileiro,Primeira caminhonete super-híbrida do mercado ...,2024-12-13T11:00:04Z,https://veja.abril.com.br/economia/a-mais-nova...,Abril.com.br
7,Ações do BB caem 3% após “bandeira laranja” no...,Os resultados do Banco do Brasil (BBAS3) no te...,2024-11-14T14:32:13Z,https://dinheirama.com/acoes-do-bb-caem-3-apos...,Dinheirama.com
8,"‘O mundo precisa do Brasil’, diz Roberto Rodri...","Na avaliação de Rodrigues, agronegócio brasil...",2024-12-13T09:00:47Z,https://veja.abril.com.br/brasil/o-mundo-preci...,Abril.com.br
9,Lula e Xi Jinping assinam 37 acordos de cooper...,"São mais de 15 áreas de atuação, incluindo int...",2024-11-20T18:03:01Z,https://veja.abril.com.br/mundo/lula-e-xi-jinp...,Abril.com.br


## ETL

In [19]:
# Creating a dataset copy so we don't modify the original one
df_copy = df.copy()

In [20]:
df_copy.tail()

,Título,Descrição,Data de Publicação,URL,Fonte
95,"Com agronegócio, Roraima lidera crescimento do...","Roraima alcançou R$ 21,1 bilhões, registrando ...",2024-11-18T17:46:54Z,https://www.metropoles.com/brasil/com-agronego...,Metropoles.com
96,None,Como a demanda global por soja e carne bovina ...,2024-12-04T14:35:57Z,https://www.uol.com.br/ecoa/ultimas-noticias/2...,Uol.com.br
97,FGV: Exclusão de carnes da cesta básica pode e...,Uma eventual exclusão das carnes da cesta bási...,2024-11-25T16:49:54Z,https://dinheirama.com/fgv-exclusao-de-carnes-...,Dinheirama.com
98,Joe Biden em visita histórica a Angola para di...,"<p>O Presidente dos Estados Unidos, Joe Biden,...",2024-12-05T22:57:00Z,https://expresso.pt/semanario/primeiro/em-dest...,Expresso.pt
99,UberHub consolida Uberlândia como polo de inov...,Startupi\n\nUberHub consolida Uberlândia como ...,2024-12-10T16:06:52Z,https://startupi.com.br/uberhub-consolida-uber...,Startupi.com.br


In [21]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Título              84 non-null     object
 1   Descrição           100 non-null    object
 2   Data de Publicação  100 non-null    object
 3   URL                 100 non-null    object
 4   Fonte               100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [15]:
df_copy.isna().sum()

Título                16
Descrição              0
Data de Publicação     0
URL                    0
Fonte                  0
dtype: int64


In [17]:
# Exibir linhas com nulos, como os nuloes estão na coluna TITULO não precisaremos remove-los.
# Displaying rows with null data, as the nulls are in the TITLE column we will not need to remove them.
linhas_com_nulos = df_copy[df_copy.isnull().any(axis=1)]
linhas_com_nulos


,Título,Descrição,Data de Publicação,URL,Fonte
28,None,"O presidente Lula (PT) e o presidente chinês, ...",2024-11-21T00:11:46Z,https://economia.uol.com.br/noticias/redacao/2...,Uol.com.br
34,None,O agronegócio brasileiro desconfia que o gover...,2024-11-25T22:26:20Z,https://noticias.uol.com.br/colunas/raquel-lan...,Uol.com.br
36,None,Quem pensa na renda fixa logo associa esses in...,2024-12-12T20:12:37Z,https://economia.uol.com.br/mais/ultimas-notic...,Uol.com.br
40,None,Lideranças do agronegócio estão indignadas com...,2024-11-24T12:45:11Z,https://economia.uol.com.br/noticias/estadao-c...,Uol.com.br
44,None,Lula cutucou o protecionismo nesta quarta (27)...,2024-11-27T16:31:04Z,https://noticias.uol.com.br/politica/ultimas-n...,Uol.com.br
46,None,"Por causa do surgimento de fatos novos, o Mini...",2024-11-20T18:16:17Z,https://noticias.uol.com.br/politica/ultimas-n...,Uol.com.br
47,None,Pesquisa do PageGroup identifica profissões pr...,2024-12-12T19:24:00Z,https://economia.uol.com.br/empregos-e-carreir...,Uol.com.br
62,None,"Para quem procura por pechinchas no mercado, e...",2024-11-29T18:49:25Z,https://economia.uol.com.br/colunas/katherine-...,Uol.com.br
66,None,Goiânia tem acumulado diversos títulos nos últ...,2024-11-24T08:55:45Z,https://noticias.uol.com.br/politica/ultimas-n...,Uol.com.br
67,None,Coautor da primeira lei estadual do Brasil que...,2024-12-12T19:13:06Z,https://noticias.uol.com.br/ultimas-noticias/r...,Uol.com.br
